# **Run each cell top to down in sequence**

In [ ]:
!sudo apt-get install tesseract-ocr
!pip install pytesseract
!pip install ultralytics==8.0.20

import cv2
from google.colab.patches import cv2_imshow
import pytesseract
from PIL import Image
import numpy as np
import re
import ultralytics
from ultralytics import YOLO
import torch
from pytesseract import Output
import imutils

In [1]:
def show_image(image):
  resized = imutils.resize(image, width=840)
  cv2_imshow(resized)

def correct_rotation(image):
  rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  results = pytesseract.image_to_osd(rgb, output_type=Output.DICT)
  rotated = imutils.rotate_bound(image, angle=results["rotate"])
  return rotated


def crop_text_area(image):
  original = image.copy()
  # *********** uncomment below lines if you need this fucntion for your input image

  # original = correct_rotation(original)
  # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  # blur = cv2.GaussianBlur(gray, (5, 5), 0)
  # thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

  # horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30,1)) # adjust value as per your input
  # detected_lines = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=3)
  # cnts = cv2.findContours(detected_lines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  # cnts = cnts[0] if len(cnts) == 2 else cnts[1]
  # for c in cnts:
  #     cv2.drawContours(thresh, [c], -1, 0, -1)

  # vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,26)) # adjust value as per your input
  # dilate = cv2.dilate(thresh, vertical_kernel, iterations=5)
  # cnts, _ = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]
  # cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:-1]
  # for c in cnts:
  #     x,y,w,h = cv2.boundingRect(c)
  #     cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 4)
  #     ROI = original[y:y+h, x:x+w]
  #     break
  #return ROI
  return original # comment this


def get_erode_image_for_OCR(image):
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  blur_image = cv2.medianBlur(gray_image,3)
  threshold_image = cv2.threshold(blur_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
  kernel = np.ones((3,3),np.uint8)
  erode_image = cv2.erode(threshold_image, kernel, iterations = 1)
  return erode_image



def get_image_width(image):
  return image.shape[1]



def get_OCR_data_filtered_by_RE(erode_image, draw_on_image):
  copy_of_draw_on_image = draw_on_image.copy()
  ocr_data = pytesseract.image_to_data(erode_image, lang='eng', output_type=Output.DICT, config='--oem 3 --psm 4')
  ocr_n_boxes = len(ocr_data['text'])
  number_pattern = '^\.?\d{1,2}\.'
  filtered_ocr_data = []

  for i in range(ocr_n_boxes):
    if int(ocr_data['conf'][i]) > 40:
    	if re.match(number_pattern, ocr_data['text'][i]):
          (x, y, w, h) = (ocr_data['left'][i], ocr_data['top'][i], ocr_data['width'][i], ocr_data['height'][i])
          filtered_ocr_data.append([ocr_data['left'][i], ocr_data['top'][i],int(re.match(number_pattern, ocr_data['text'][i]).group()[:-1])])
          img = cv2.rectangle(copy_of_draw_on_image, (x, y), (x + w, y + h), (241, 153, 78), 3)

  show_image(copy_of_draw_on_image)
  return filtered_ocr_data



def sort_left_right_top_bottom(coordinates, image, yolo_coordinates=False):
  left_half_axis = []
  right_half_axis = []
  width = image.shape[1]
  sorted_coordinates = []
  for coord in coordinates:
      if coord[0] < (width//2)-100:
          left_half_axis.append(coord)
      else:
          right_half_axis.append(coord)

  left_half_axis.sort(key=lambda x: x[1])
  right_half_axis.sort(key=lambda x: x[1])
  if not yolo_coordinates:
    sorted_coordinates.extend([left_half_axis])
    sorted_coordinates.extend([right_half_axis])
  else:
    sorted_coordinates = left_half_axis + right_half_axis
  return sorted_coordinates


In [ ]:
def xyxy2xywh(x):
    assert x.shape[-1] == 4, f'input shape last dimension expected 4 but input shape is {x.shape}'
    y = torch.empty_like(x) if isinstance(x, torch.Tensor) else np.empty_like(x)
    y[..., 0] = x[..., 0]
    y[..., 1] = x[..., 1]
    y[..., 2] = x[..., 2] - x[..., 0]
    y[..., 3] = x[..., 3] - x[..., 1]
    return y


def predict_and_YOLO_bbx(image):
  copied_image = image.copy()
  labels = {0: "Cross", 1: "Fill", 2: "Strike", 3: "Tick"}
  model = YOLO("/content/drive/MyDrive/YOLO_170img_150epoch_06Nov/train/weights/best.pt")
  results = model.predict(copied_image)

  selection=[]
  selection_sorted=[]

  for result in results:
      boxes = result.boxes.cpu().numpy()
      class_ids = result.boxes.cls.cpu().numpy()

      for box, class_id in zip(boxes, class_ids):
          if class_id != 0:
            r = box.xyxy[0].astype(int)
            selection.append(r.tolist())
            cv2.rectangle(copied_image, r[:2], r[2:], (0, 255, 0), 3)
          else:
            cross = box.xyxy[0].astype(int)
            cv2.rectangle(copied_image, cross[:2], cross[2:], (0, 255, 255), 3)

  selection_sorted = sort_left_right_top_bottom(selection,copied_image, yolo_coordinates=True)
  show_image(copied_image)
  return selection_sorted




def get_answer(image, yolo_bounding_boxes, sorted_ocr_boxes, student_script=False):
  yolo_bounding_boxes =  [box[0].tolist() for box in [xyxy2xywh(np.array([box])) for box in yolo_bounding_boxes]]
  answers = {}
  thresh = 255 - cv2.threshold(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
  column_1_ocr_data_len = len(sorted_ocr_boxes[0])
  column_2_ocr_data_len = len(sorted_ocr_boxes[1])
  ocr_index = 1
  go_to_2nd_column = False
  for i in range(len(yolo_bounding_boxes)):
    x,y,w,h = yolo_bounding_boxes[i][:4]
    x,y,w,h = x+w+6,y-5,1000,h+6
    ROI = thresh[y:y+h,x:x+w]
    text_data = pytesseract.image_to_string(ROI, lang='eng',config='--oem 3 --psm 6')
    alphanumeric_data = re.sub(r'[^a-zA-Z0-9]', '', text_data)
    first_20_char = alphanumeric_data[:20]

    question_number= None
    if not go_to_2nd_column:
      while not go_to_2nd_column and question_number is None and ocr_index <= column_1_ocr_data_len-1:
        if y > sorted_ocr_boxes[0][ocr_index - 1][1] and y < sorted_ocr_boxes[0][ocr_index][1]:
            question_number = sorted_ocr_boxes[0][ocr_index - 1][2]
        elif y > sorted_ocr_boxes[0][ocr_index][1] and ocr_index + 1 < column_1_ocr_data_len:
            ocr_index += 1
        elif y > sorted_ocr_boxes[0][ocr_index][1] and ocr_index == column_1_ocr_data_len - 1:
            question_number = sorted_ocr_boxes[0][ocr_index][2]
        elif ocr_index == column_1_ocr_data_len - 1:
            ocr_index = 0
            go_to_2nd_column = True
        else:
          ocr_index += 1

    if go_to_2nd_column:
      while question_number is None and ocr_index <= column_2_ocr_data_len-1:
        if y > sorted_ocr_boxes[1][ocr_index - 1][1] and y < sorted_ocr_boxes[1][ocr_index][1]:
            question_number = sorted_ocr_boxes[1][ocr_index - 1][2]
        elif y > sorted_ocr_boxes[1][ocr_index][1] and ocr_index + 1 < column_2_ocr_data_len:
            ocr_index += 1
        elif y > sorted_ocr_boxes[1][ocr_index][1] and ocr_index == column_2_ocr_data_len - 1:
            question_number = sorted_ocr_boxes[1][ocr_index][2]
        else:
          ocr_index += 1

    if student_script:
      if question_number in answers:
        answers[question_number].append((first_20_char, i))
      else:
        answers[question_number] = [(first_20_char, i)]

    else:
      if question_number in answers:
        answers[question_number].append(first_20_char)
      else:
        answers[question_number] = [first_20_char]

  return answers



def compare_answers_and_get_mark(teacher_answers, student_answers):
    student_marks = 0
    unmacthed_yolo_index =[]
    wrong_answers = []
    correct_answer = []
    not_answered = []

    for key, teacher_value in teacher_answers.items():
        if key in student_answers:
            student_value_tuple = student_answers[key]
            unmatched_index_list = []

            for student_value, yolo_index in student_value_tuple:
                if student_value not in teacher_value:
                    unmatched_index_list.append(yolo_index)

            if unmatched_index_list:
                unmacthed_yolo_index.extend(unmatched_index_list)
                wrong_answers.append(key)
            elif len(student_value_tuple) == len(teacher_value) and not unmatched_index_list:
                student_marks += 1
                correct_answer.append(key)
            else:
              wrong_answers.append(key)
        else:
          not_answered.append(key)

    print("Student marks:", student_marks)
    print("Correct answer (Qstn no):", correct_answer)
    print("Wrong answer (Qstn no):", wrong_answers)
    print("Not answered (Qstn no)", not_answered)
    return unmacthed_yolo_index


def draw_red_mark_on_yolo_image(image, all_yolo_coordinates, unmatched_yolo_index):
  copied_image = image.copy()
  for k in unmatched_yolo_index:
    r = all_yolo_coordinates[k]
    cv2.rectangle(copied_image, r[:2], r[2:], (0, 0, 255), 3)
  show_image(copied_image)

In [ ]:
# @title Teacher
teacher_original_image = crop_text_area(cv2.imread('teacher image path'))
teacher_erode_image = get_erode_image_for_OCR(teacher_original_image)
t_ocr_data = get_OCR_data_filtered_by_RE(teacher_erode_image, teacher_original_image)
teacher_ocr_data = sort_left_right_top_bottom(t_ocr_data, teacher_original_image)

print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Sorted OCR data:")
for x in teacher_ocr_data:
  print(x)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

In [ ]:
# @title Teacher
teacher_yolo_coordinates = predict_and_YOLO_bbx(teacher_original_image)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Yolo coordinates (selected annotation only):")
for x in teacher_yolo_coordinates:
  print(x)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

In [ ]:
# @title Teacher
teacher_answers = get_answer(teacher_original_image, teacher_yolo_coordinates, teacher_ocr_data)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Answer strings for each question {Qstn:Ans pair}")
for x in teacher_answers:
  print(x, ":", teacher_answers[x])
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

In [ ]:
# @title Student
student_original_image = crop_text_area(cv2.imread('student image path'))
student_erode_image = get_erode_image_for_OCR(student_original_image)

s_ocr_data = get_OCR_data_filtered_by_RE(student_erode_image, student_original_image)
student_ocr_data = sort_left_right_top_bottom(s_ocr_data, student_original_image)

print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Sorted OCR data:")
for x in student_ocr_data:
  print(x)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

In [ ]:
# @title Student
student_yolo_coordinates = predict_and_YOLO_bbx(student_original_image)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Yolo coordinates (selected annotation only):")
for x in student_yolo_coordinates:
  print(x)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

In [ ]:
# @title Student
student_answers = get_answer(student_original_image, student_yolo_coordinates, student_ocr_data, student_script=True)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Answer strings for each question {Qstn:Ans pair}")
for x in student_answers:
  print(x, ":", student_answers[x])
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

In [ ]:
unmatched_yolo_index = compare_answers_and_get_mark(teacher_answers, student_answers)
draw_red_mark_on_yolo_image(student_original_image, student_yolo_coordinates, unmatched_yolo_index)

# Use this portion to run all the functions in a single click (Multiprocessing)

In [ ]:
# modify this to handle both input image

# from multiprocessing import Process, Queue

# def getOCR(student_original_image, outputOCR):
#   student_erode_image = get_erode_image_for_OCR(student_original_image)
#   s_ocr_data = get_OCR_data_filtered_by_RE(student_erode_image, student_original_image)
#   student_ocr_data = sort_left_right_top_bottom(s_ocr_data, student_original_image)
#   outputOCR.put(student_ocr_data)

# def getYOLO(student_original_image, outputYOLO):
#     student_yolo_coordinates = predict_and_YOLO_bbx(student_original_image)
#     outputYOLO.put(student_yolo_coordinates)

In [ ]:
# student_original_image = crop_text_area(cv2.imread('student image path '))

# if __name__=='__main__':
#     outputOCR = Queue()
#     outputYOLO = Queue()

#     student_ocr_data = Process(target = getOCR(student_original_image, outputOCR))
#     student_ocr_data.start()
#     student_yolo_coordinates = Process(target = getYOLO(student_original_image, outputYOLO))
#     student_yolo_coordinates.start()

#     student_answers = get_answer(student_original_image, outputYOLO.get(), outputOCR.get(), student_script=True)
#     unmatched_yolo_index = compare_answers_and_get_mark(teacher_answers, student_answers)
